This notebook detects 2 class objects.
- class1: helmet without impact
- class2: helmet with impact

Object Detection part is based on [EfficientDet notebook](https://www.kaggle.com/shonenkov/training-efficientdet) for [global wheat detection competition](https://www.kaggle.com/c/global-wheat-detection) by [shonenkov](https://www.kaggle.com/shonenkov), which is using [github repos efficientdet-pytorch](https://github.com/rwightman/efficientdet-pytorch) by [@rwightman](https://www.kaggle.com/rwightman).

Inference part can be foud [here](https://www.kaggle.com/its7171/2class-object-detection-inference/).

In [1]:
# !pip install ./timm-0.1.26-py3-none-any.whl
# !tar xfz ./pkgs.tgz
# for pytorch1.6
# cmd = "sed -i -e 's/ \/ / \/\/ /' timm-efficientdet-pytorch/effdet/bench.py"
# !$cmd

# Import

In [2]:
import sys
# sys.path.insert(0, "timm-efficientdet-pytorch")
sys.path.insert(0, "efficientdet-pytorch-master")
sys.path.insert(0, "omegaconf")

import torch
import os
from datetime import datetime
import time
import random
import cv2
import pandas as pd
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from glob import glob
import pandas as pd
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain, DetBenchPredict
from effdet.efficientdet import HeadNet
from tqdm import tqdm
from IPython.core.debugger import set_trace
import warnings
from scipy.optimize import linear_sum_assignment

warnings.filterwarnings("ignore")

SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(SEED)

IMG_H = 512
IMG_W = 512

## Utils

In [3]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [4]:
def iou(bbox1, bbox2):
    bbox1 = [float(x) for x in bbox1]
    bbox2 = [float(x) for x in bbox2]

    (x0_1, y0_1, x1_1, y1_1) = bbox1
    (x0_2, y0_2, x1_2, y1_2) = bbox2

    # get the overlap rectangle
    overlap_x0 = max(x0_1, x0_2)
    overlap_y0 = max(y0_1, y0_2)
    overlap_x1 = min(x1_1, x1_2)
    overlap_y1 = min(y1_1, y1_2)

    # check if there is an overlap
    if overlap_x1 - overlap_x0 <= 0 or overlap_y1 - overlap_y0 <= 0:
            return 0

    # if yes, calculate the ratio of the overlap to each ROI size and the unified size
    size_1 = (x1_1 - x0_1) * (y1_1 - y0_1)
    size_2 = (x1_2 - x0_2) * (y1_2 - y0_2)
    size_intersection = (overlap_x1 - overlap_x0) * (overlap_y1 - overlap_y0)
    size_union = size_1 + size_2 - size_intersection

    return size_intersection / size_union

def precision_calc(gt_boxes, pred_boxes):
    cost_matix = np.ones((len(gt_boxes), len(pred_boxes)))
    for i, box1 in enumerate(gt_boxes):
        for j, box2 in enumerate(pred_boxes):
            dist = abs(box1[0]-box2[0])
            if dist > 4:
                continue
            iou_score = iou(box1[1:], box2[1:])

            if iou_score < 0.35:
                continue
            else:
                cost_matix[i,j]=0

    row_ind, col_ind = linear_sum_assignment(cost_matix)
    fn = len(gt_boxes) - row_ind.shape[0]
    fp = len(pred_boxes) - col_ind.shape[0]
    tp=0
    for i, j in zip(row_ind, col_ind):
        if cost_matix[i,j]==0:
            tp+=1
        else:
            fp+=1
            fn+=1
    return tp, fp, fn

def f1_calc(data):
    data_dict = {}
    for key, value in data.items():
        video = "_".join(key.split(".")[0].split("_")[:-1])
        frame = int(key.split(".")[0].split("_")[-1])
        
        if video not in data_dict:
            data_dict[video] = ([], [])
        
        gt_boxes, pred_boxes = [], []
        
        for box in value[0]:
            gt_boxes.append([frame] + box.tolist())
            
        for box in value[1]:
            pred_boxes.append([frame] + box.tolist())
            
        data_dict[video][0].extend(gt_boxes)
        data_dict[video][1].extend(pred_boxes)
    
    
    ftp, ffp, ffn = [], [], []
    
    for video, data in data_dict.items():
        gt_boxes = data[0]
        pred_boxes = data[1]
        tp, fp, fn = precision_calc(gt_boxes, pred_boxes)
        ftp.append(tp)
        ffp.append(fp)
        ffn.append(fn)

    tp = np.sum(ftp)
    fp = np.sum(ffp)
    fn = np.sum(ffn)
    precision = tp / (tp + fp + 1e-6)
    recall =  tp / (tp + fn +1e-6)
    f1_score = 2*(precision*recall)/(precision+recall+1e-6)
    
    print(f'TP: {tp}, FP: {fp}, FN: {fn}, PRECISION: {precision:.4f}, RECALL: {recall:.4f}, F1 SCORE: {f1_score}')
    return f1_score

## Config

In [5]:
class TrainGlobalConfig:
    num_workers = 8
    batch_size = 4
    n_epochs = 7
    lr = 0.0002
    score_threshold = 0.4
    folder = 'effdet5-models'
    verbose = True
    verbose_step = 1
    step_scheduler = False
    validation_scheduler = True
    
    SchedulerClass = torch.optim.lr_scheduler.ReduceLROnPlateau
    scheduler_params = dict(
        mode='min',
        factor=0.5,
        patience=1,
        verbose=False, 
        threshold=0.0001,
        threshold_mode='abs',
        cooldown=0, 
        min_lr=1e-8,
        eps=1e-08
    )

## Data Preparation

In [6]:
video_labels = pd.read_csv('/home/thinh/nfl/train_labels.csv').fillna(0)
video_labels.head()

,gameKey,playID,view,video,frame,label,left,width,top,height,impact,impactType,confidence,visibility
0,57583,82,Endzone,57583_000082_Endzone.mp4,1,H30,629,19,40,24,0.0,0,0.0,0.0
1,57583,82,Endzone,57583_000082_Endzone.mp4,1,V72,443,22,344,16,0.0,0,0.0,0.0
2,57583,82,Endzone,57583_000082_Endzone.mp4,1,V86,871,21,359,17,0.0,0,0.0,0.0
3,57583,82,Endzone,57583_000082_Endzone.mp4,1,V74,771,19,345,15,0.0,0,0.0,0.0
4,57583,82,Endzone,57583_000082_Endzone.mp4,1,V34,549,26,461,20,0.0,0,0.0,0.0


In [7]:
video_labels_with_impact = video_labels[video_labels['impact'] > 0]
for row in tqdm(video_labels_with_impact[['video','frame','label']].values):
    frames = np.array([-4,-3,-2,-1,1,2,3,4])+row[1]
    video_labels.loc[(video_labels['video'] == row[0]) 
                                 & (video_labels['frame'].isin(frames))
                                 & (video_labels['label'] == row[2]), 'impact'] = 1
    
video_labels['image_name'] = video_labels['video'].str.replace('.mp4', '') + '_' + video_labels['frame'].astype(str) + '.png'
video_labels = video_labels[video_labels.groupby('image_name')['impact'].transform("sum") > 0].reset_index(drop=True)
video_labels['impact'] = video_labels['impact'].astype(int)+1
video_labels['x'] = video_labels['left']
video_labels['y'] = video_labels['top']
video_labels['w'] = video_labels['width']
video_labels['h'] = video_labels['height']
video_labels.head()

100%|██████████| 2291/2291 [02:59<00:00, 12.78it/s]


,gameKey,playID,view,video,frame,label,left,width,top,height,impact,impactType,confidence,visibility,image_name,x,y,w,h
0,57583,82,Endzone,57583_000082_Endzone.mp4,34,V73,655,21,331,15,1,0,0.0,0.0,57583_000082_Endzone_34.png,655,331,21,15
1,57583,82,Endzone,57583_000082_Endzone.mp4,34,H99,583,21,312,30,2,0,0.0,0.0,57583_000082_Endzone_34.png,583,312,21,30
2,57583,82,Endzone,57583_000082_Endzone.mp4,34,V15,1069,22,301,20,1,0,0.0,0.0,57583_000082_Endzone_34.png,1069,301,22,20
3,57583,82,Endzone,57583_000082_Endzone.mp4,34,H97,402,21,313,29,1,0,0.0,0.0,57583_000082_Endzone_34.png,402,313,21,29
4,57583,82,Endzone,57583_000082_Endzone.mp4,34,V72,445,21,328,16,1,0,0.0,0.0,57583_000082_Endzone_34.png,445,328,21,16


In [8]:
video_names = np.random.permutation(video_labels.video.unique())
valid_video_len = int(len(video_names)*0.2)
video_valid = video_names[:valid_video_len]
video_train = video_names[valid_video_len:]
images_valid = video_labels[ video_labels.video.isin(video_valid)].image_name.unique()
images_train = video_labels[~video_labels.video.isin(video_valid)].image_name.unique()

In [9]:
# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# df_folds = video_labels[['image_name']].copy()
# df_folds.loc[:, 'bbox_count'] = 1
# df_folds = df_folds.groupby('image_name').count()
# df_folds.loc[:, 'video'] = video_labels[['image_name', 'video']].groupby('image_name').min()['video']
# df_folds.loc[:, 'stratify_group'] = np.char.add(
#     df_folds['video'].values.astype(str),
#     df_folds['bbox_count'].apply(lambda x: f'_{x // 20}').values.astype(str),
# )

# df_folds.loc[:, 'fold'] = 0
# for fold_number, (train_index, val_index) in enumerate(skf.split(X=df_folds.index, y=df_folds['stratify_group'])):
#     df_folds.loc[df_folds.iloc[val_index].index, 'fold'] = fold_number
    
# df_folds.head()

In [10]:
# export frame in video, only frame with impact
# def mk_images(video_name, video_labels, video_dir, out_dir, only_with_impact=True):
#     video_path=f"{video_dir}/{video_name}"
#     video_name = os.path.basename(video_path)
#     vidcap = cv2.VideoCapture(video_path)
    
#     if only_with_impact:
#         boxes_all = video_labels.query("video == @video_name")
#         print(video_path, boxes_all[boxes_all.impact > 1.0].shape[0])
#     else:
#         print(video_path)
        
#     frame = 0
#     while True:
#         it_worked, img = vidcap.read()
#         if not it_worked:
#             break
#         frame += 1
#         if only_with_impact:
#             boxes = video_labels.query("video == @video_name and frame == @frame")
#             boxes_with_impact = boxes[boxes.impact > 1.0]
#             if boxes_with_impact.shape[0] == 0:
#                 continue
                
#         image_path = f'{out_dir}/{video_name}'.replace('.mp4',f'_{frame}.png')
#         _ = cv2.imwrite(image_path, img)

In [11]:
# uniq_video = video_labels.video.unique()
# video_dir = '/home/thinh/nfl/train'
# out_dir = '/home/thinh/nfl/train_images'
# !mkdir -p $out_dir
# for video_name in uniq_video:
#     mk_images(video_name, video_labels, video_dir, out_dir)

## Albumentations

In [12]:
def get_train_transforms():
    return A.Compose(
        [
            A.HorizontalFlip(p=0.5),
            A.Resize(height=IMG_H, width=IMG_W, p=1),
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']
        )
    )

def get_valid_transforms():
    return A.Compose(
        [
            A.Resize(height=IMG_H, width=IMG_W, p=1.0),
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']
        )
    )

# def get_train_transforms():
#     return A.Compose(
#         [
#             A.RandomSizedCrop(min_max_height=(500, 720), height=720, width=720, p=0.5),
#             A.OneOf([
#                 A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit= 0.2, 
#                                      val_shift_limit=0.2, p=0.9),
#                 A.RandomBrightnessContrast(brightness_limit=0.2, 
#                                            contrast_limit=0.2, p=0.9),
#             ], p=0.9),
#             A.ToGray(p=0.01),
#             A.HorizontalFlip(p=0.5),
#             A.VerticalFlip(p=0.5),
#             A.RandomRotate90(p=0.5),
#             A.Transpose(p=0.5),
#             A.JpegCompression(quality_lower=85, quality_upper=95, p=0.2),
#             A.OneOf([
#                 A.Blur(blur_limit=3, p=1.0),
#                 A.MedianBlur(blur_limit=3, p=1.0)
#             ],p=0.1),
#             A.Resize(height=IMG_H, width=IMG_W, p=1),
#             A.Cutout(num_holes=8, max_h_size=64, max_w_size=64, fill_value=0, p=0.5),
#             ToTensorV2(p=1.0),
#         ], 
#         p=1.0, 
#         bbox_params=A.BboxParams(
#             format='pascal_voc',
#             min_area=0, 
#             min_visibility=0,
#             label_fields=['labels']
#         )
#     )

# def get_valid_transforms():
#     return A.Compose(
#         [
#             A.Resize(height=IMG_H, width=IMG_W, p=1.0),
#             ToTensorV2(p=1.0),
#         ], 
#         p=1.0, 
#         bbox_params=A.BboxParams(
#             format='pascal_voc',
#             min_area=0, 
#             min_visibility=0,
#             label_fields=['labels']
#         )
#     )

## Dataset

In [13]:
TRAIN_ROOT_PATH = 'train_images'

class DatasetRetriever(Dataset):

    def __init__(self, marking, image_ids, transforms=None, test=False):
        super().__init__()

        self.image_ids = image_ids
        self.marking = marking
        self.transforms = transforms
        self.test = test

    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        
#         if self.test or random.random() > 0.33:
        image, boxes, labels = self.load_image_and_boxes(index)
#         elif random.random() > 0.5:
#             image, boxes, labels = self.load_cutmix_image_and_boxes(index)
#         else:
#             image, boxes, labels = self.load_mixup_image_and_boxes(index)
                
        target = {}
        target['boxes'] = boxes
        target['labels'] = torch.tensor(labels)
        target['image_id'] = torch.tensor([index])

        if self.transforms:
            for i in range(10):
                sample = self.transforms(**{
                    'image': image,
                    'bboxes': target['boxes'],
                    'labels': labels
                })
                if len(sample['bboxes']) > 0:
                    image = sample['image']
                    target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)
                    target['boxes'][:,[0,1,2,3]] = target['boxes'][:,[1,0,3,2]]  #yxyx: be warning
                    break
                    
        return image, target, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

    def load_image_and_boxes(self, index):
        image_id = self.image_ids[index]
        #print(f'{TRAIN_ROOT_PATH}/{image_id}')
        image = cv2.imread(f'/home/thinh/nfl/{TRAIN_ROOT_PATH}/{image_id}', cv2.IMREAD_COLOR).copy().astype(np.float32)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        records = self.marking[self.marking['image_name'] == image_id]
        boxes = records[['x', 'y', 'w', 'h']].values
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
        labels = records['impact'].values
        return image, boxes, labels
    
    def load_mixup_image_and_boxes(self, index):
        image, boxes, labels = self.load_image_and_boxes(index)
        r_image, r_boxes, r_labels = self.load_image_and_boxes(random.randint(0, self.image_ids.shape[0] - 1))
        return (image+r_image)/2, np.vstack((boxes, r_boxes)).astype(np.int32), np.concatenate((labels, r_labels))
    

    def load_cutmix_image_and_boxes(self, index, imsize=720):
        """ 
        This implementation of cutmix author:  https://www.kaggle.com/nvnnghia 
        Refactoring and adaptation: https://www.kaggle.com/shonenkov
        """
        w, h = imsize, imsize
        s = imsize // 2
    
        xc, yc = [int(random.uniform(imsize * 0.25, imsize * 0.75)) for _ in range(2)]  # center x, y
        indexes = [index] + [random.randint(0, self.image_ids.shape[0] - 1) for _ in range(3)]

        result_image = np.full((imsize, imsize, 3), 1, dtype=np.float32)
        result_boxes = []
        result_labels = np.array([], dtype=np.int)

        for i, index in enumerate(indexes):
            image, boxes, labels = self.load_image_and_boxes(index)
            if i == 0:
                x1a, y1a, x2a, y2a = max(xc - w, 0), max(yc - h, 0), xc, yc  # xmin, ymin, xmax, ymax (large image)
                x1b, y1b, x2b, y2b = w - (x2a - x1a), h - (y2a - y1a), w, h  # xmin, ymin, xmax, ymax (small image)
            elif i == 1:  # top right
                x1a, y1a, x2a, y2a = xc, max(yc - h, 0), min(xc + w, s * 2), yc
                x1b, y1b, x2b, y2b = 0, h - (y2a - y1a), min(w, x2a - x1a), h
            elif i == 2:  # bottom left
                x1a, y1a, x2a, y2a = max(xc - w, 0), yc, xc, min(s * 2, yc + h)
                x1b, y1b, x2b, y2b = w - (x2a - x1a), 0, max(xc, w), min(y2a - y1a, h)
            elif i == 3:  # bottom right
                x1a, y1a, x2a, y2a = xc, yc, min(xc + w, s * 2), min(s * 2, yc + h)
                x1b, y1b, x2b, y2b = 0, 0, min(w, x2a - x1a), min(y2a - y1a, h)
            result_image[y1a:y2a, x1a:x2a] = image[y1b:y2b, x1b:x2b]
            padw = x1a - x1b
            padh = y1a - y1b

            boxes[:, 0] += padw
            boxes[:, 1] += padh
            boxes[:, 2] += padw
            boxes[:, 3] += padh

            result_boxes.append(boxes)
            result_labels = np.concatenate((result_labels, labels))

        result_boxes = np.concatenate(result_boxes, 0)
        np.clip(result_boxes[:, 0:], 0, 2 * s, out=result_boxes[:, 0:])
        result_boxes = result_boxes.astype(np.int32)
        index_to_use = np.where((result_boxes[:,2]-result_boxes[:,0])*(result_boxes[:,3]-result_boxes[:,1]) > 0)
        result_boxes = result_boxes[index_to_use]
        result_labels = result_labels[index_to_use]
        
        return result_image, result_boxes, result_labels

## Fitter

In [14]:
class Fitter:
    
    def __init__(self, model, device, config):
        self.config = config
        self.epoch = 0

        self.base_dir = f'/home/thinh/nfl/{config.folder}'
        if not os.path.exists(self.base_dir):
            os.makedirs(self.base_dir)
        
        self.log_path = f'{self.base_dir}/log.txt'
        self.best_summary_loss = 10**5

        self.model = model
        self.device = device

        param_optimizer = list(self.model.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ] 

        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=config.lr)
        self.scheduler = config.SchedulerClass(self.optimizer, **config.scheduler_params)
        self.log(f'Fitter prepared. Device is {self.device}')

    def fit(self, train_loader, validation_loader, fold):
        scaler = torch.cuda.amp.GradScaler()
        
        for e in range(self.config.n_epochs):
            if self.config.verbose:
                lr = self.optimizer.param_groups[0]['lr']
                timestamp = datetime.utcnow().isoformat()
                self.log(f'\n{timestamp}\nLR: {lr}')

            t = time.time()
            summary_loss = self.train_one_epoch(train_loader, scaler)

            self.log(f'[RESULT]: Train. Epoch: {self.epoch}, summary_loss: {summary_loss.avg:.5f}, time: {(time.time() - t):.5f}')
            self.save(f'{self.base_dir}/tito-last-checkpoint.bin')

            t = time.time()
            summary_loss = self.validation(validation_loader)

            self.log(f'[RESULT]: Val. Epoch: {self.epoch}, summary_loss: {summary_loss.avg:.5f}, time: {(time.time() - t):.5f}')
            if summary_loss.avg < self.best_summary_loss:
                self.best_summary_loss = summary_loss.avg
                self.model.eval()
                self.save(f'{self.base_dir}/tito-best-checkpoint-epoch{str(self.epoch).zfill(3)}-fold{str(fold)}.bin')
#                 for path in sorted(glob(f'{self.base_dir}/best-checkpoint-*epoch.bin'))[:-3]:
#                     os.remove(path)

            if self.config.validation_scheduler:
                self.scheduler.step(metrics=summary_loss.avg)

            self.epoch += 1

    def validation(self, val_loader):
        valid_data = {}
        self.model.eval()
        summary_loss = AverageMeter()
        t = time.time()
        
        for step, (images, targets, image_ids) in enumerate(val_loader):
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    print(
                        f'Val Step {step}/{len(val_loader)}, ' + \
                        f'summary_loss: {summary_loss.avg:.5f}, ' + \
                        f'time: {(time.time() - t):.5f}', end='\r'
                    )
                    
            with torch.no_grad():
                images = torch.stack(images)
                batch_size = images.shape[0]
                images = images.to(self.device).float()
                boxes = [target['boxes'].to(self.device).float() for target in targets]
                labels = [target['labels'].to(self.device).float() for target in targets]
                
                target_res = {}
                target_res['bbox'] = boxes
                target_res['cls'] = labels 
                target_res['img_scale'] = torch.tensor([1]*images.shape[0]).float().to(self.device)
                target_res['img_size'] = torch.tensor(images.shape[2:]).repeat(images.shape[0], 1).to(self.device)
                
                outputs = self.model(images, target_res)
#                 loss, _, _ = self.model(images, boxes, labels)
                loss = outputs['loss']
                det = outputs['detections']
                
                for i in range(images.shape[0]):
                    pre_boxes = det[i].detach().cpu().numpy()[:,:4]    
                    pre_scores = det[i].detach().cpu().numpy()[:,4]
                    pre_label = det[i].detach().cpu().numpy()[:,5]
                    
                    # using only label = 2
                    pre_indexes = np.where((pre_scores > self.config.score_threshold) & (pre_label == 2))[0]
                    
                    true_indexes = np.where(labels[i].cpu().numpy() == 2)[0]
                    valid_data[image_ids[i]] = (boxes[i][true_indexes].cpu().numpy(), pre_boxes[pre_indexes])
                
                summary_loss.update(loss.detach().item(), batch_size)
        
        val_f1 = f1_calc(valid_data)
        
        return summary_loss

    def train_one_epoch(self, train_loader, scaler):
        self.model.train()
        summary_loss = AverageMeter()
        t = time.time()
        
        for step, (images, targets, image_ids) in enumerate(train_loader):
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    print(
                        f'Train Step {step}/{len(train_loader)}, ' + \
                        f'summary_loss: {summary_loss.avg:.5f}, ' + \
                        f'time: {(time.time() - t):.5f}', end='\r'
                    )
            
            images = torch.stack(images)
            images = images.to(self.device).float()
            batch_size = images.shape[0]
            boxes = [target['boxes'].to(self.device).float() for target in targets]
            labels = [target['labels'].to(self.device).float() for target in targets]

            target_res = {}
            target_res['bbox'] = boxes
            target_res['cls'] = labels
            
            self.optimizer.zero_grad()
#             with torch.cuda.amp.autocast():
            outputs = self.model(images, target_res)
                
            loss = outputs['loss']
            loss.backward()
#             scaler.scale(loss).backward()

            summary_loss.update(loss.detach().item(), batch_size)

            self.optimizer.step()
#             scaler.step(self.optimizer)
#             scaler.update()

            if self.config.step_scheduler:
                self.scheduler.step()

        return summary_loss
    
    def save(self, path):
        self.model.eval()
        torch.save({
            'model_state_dict': self.model.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'best_summary_loss': self.best_summary_loss,
            'epoch': self.epoch,
        }, path)

    def load(self, path):
        checkpoint = torch.load(path)
        self.model.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        self.best_summary_loss = checkpoint['best_summary_loss']
        self.epoch = checkpoint['epoch'] + 1
        
    def log(self, message):
        if self.config.verbose:
            print(message)
        with open(self.log_path, 'a+') as logger:
            logger.write(f'{message}\n')

## Training

In [15]:
def collate_fn(batch):
    return tuple(zip(*batch))
    
def run_training(num_fold):
    device = torch.device('cuda:1')
    
    for fold_number in range(num_fold):
        print('Fold: {}'.format(fold_number + 1))

        train_dataset = DatasetRetriever(
#             image_ids=df_folds[df_folds['fold'] != fold_number].index.values,
            image_ids=images_train,
            marking=video_labels,
            transforms=get_train_transforms(),
            test=False,
        )

        validation_dataset = DatasetRetriever(
#             image_ids=df_folds[df_folds['fold'] == fold_number].index.values,
            image_ids=images_valid,
            marking=video_labels,
            transforms=get_valid_transforms(),
            test=True,
        )

        train_loader = torch.utils.data.DataLoader(
            train_dataset,
            batch_size=TrainGlobalConfig.batch_size,
            sampler=RandomSampler(train_dataset),
            pin_memory=False,
            drop_last=True,
            num_workers=TrainGlobalConfig.num_workers,
            collate_fn=collate_fn,
        )
        val_loader = torch.utils.data.DataLoader(
            validation_dataset, 
            batch_size=TrainGlobalConfig.batch_size,
            num_workers=TrainGlobalConfig.num_workers,
            shuffle=False,
            sampler=SequentialSampler(validation_dataset),
            pin_memory=False,
            collate_fn=collate_fn,
        )
        
        net = get_net()
        net.to(device)

        fitter = Fitter(model=net, device=device, config=TrainGlobalConfig)
        fitter.fit(train_loader, val_loader, fold_number)

In [16]:
def get_net():
#     config = get_efficientdet_config('tf_efficientdet_d5')
#     net = EfficientDet(config, pretrained_backbone=False)
#     checkpoint = torch.load('./efficientdet_d5-ef44aea8.pth')
#     net.load_state_dict(checkpoint)
#     config.num_classes = 2
#     config.image_size = 768
#     net.class_net = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=dict(eps=.001, momentum=.01))
#     return DetBenchTrain(net, config)

    config = get_efficientdet_config('tf_efficientdet_d5')

    config.image_size = [IMG_H, IMG_W]
    config.norm_kwargs=dict(eps=.001, momentum=.01)

    net = EfficientDet(config, pretrained_backbone=False)
    checkpoint = torch.load('./efficientdet_d5-ef44aea8.pth')
    net.load_state_dict(checkpoint)

    net.reset_head(num_classes=2)
    net.class_net = HeadNet(config, num_outputs=config.num_classes)

    return DetBenchTrain(net)

net = get_net()

In [17]:
run_training(num_fold=1)

Fold: 1
Fitter prepared. Device is cuda:1

2020-12-19T05:40:28.708511
LR: 0.0002
[RESULT]: Train. Epoch: 0, summary_loss: 32.99194, time: 1255.69073
TP: 0, FP: 0, FN: 3782, PRECISION: 0.0000, RECALL: 0.0000, F1 SCORE: 0.0
[RESULT]: Val. Epoch: 0, summary_loss: 0.71405, time: 85.95680

2020-12-19T06:02:52.023914
LR: 0.0002
[RESULT]: Train. Epoch: 1, summary_loss: 0.45980, time: 1281.74998
TP: 19, FP: 363, FN: 3763, PRECISION: 0.0497, RECALL: 0.0050, F1 SCORE: 0.009125673891161076
[RESULT]: Val. Epoch: 1, summary_loss: 0.49883, time: 86.68298

2020-12-19T06:25:42.197402
LR: 0.0002
[RESULT]: Train. Epoch: 2, summary_loss: 0.34238, time: 1281.28315
TP: 19, FP: 1137, FN: 3763, PRECISION: 0.0164, RECALL: 0.0050, F1 SCORE: 0.007695064664432617
[RESULT]: Val. Epoch: 2, summary_loss: 0.47902, time: 87.50339

2020-12-19T06:48:32.722592
LR: 0.0002
[RESULT]: Train. Epoch: 3, summary_loss: 0.29125, time: 1280.27032
TP: 18, FP: 1177, FN: 3764, PRECISION: 0.0151, RECALL: 0.0048, F1 SCORE: 0.007232908

In [18]:
# Fold: 1
# Fitter prepared. Device is cuda:1

# 2020-12-15T08:34:29.752664
# LR: 0.0002
# [RESULT]: Train. Epoch: 0, summary_loss: 35.16191, time: 2350.88583
# [RESULT]: Val. Epoch: 0, summary_loss: 0.59447, time: 93.20803

# 2020-12-15T09:15:15.461840
# LR: 0.0002
# [RESULT]: Train. Epoch: 1, summary_loss: 0.45924, time: 2367.97266
# [RESULT]: Val. Epoch: 1, summary_loss: 0.39323, time: 93.17049

# 2020-12-15T09:56:18.243550
# LR: 0.0002
# [RESULT]: Train. Epoch: 2, summary_loss: 0.35253, time: 2370.91510
# [RESULT]: Val. Epoch: 2, summary_loss: 0.32969, time: 93.07946

# 2020-12-15T10:37:23.930087
# LR: 0.0002
# [RESULT]: Train. Epoch: 3, summary_loss: 0.30887, time: 2370.21342
# [RESULT]: Val. Epoch: 3, summary_loss: 0.30014, time: 93.32968

# 2020-12-15T11:18:29.135952
# LR: 0.0002
# [RESULT]: Train. Epoch: 4, summary_loss: 0.27514, time: 2370.02368
# [RESULT]: Val. Epoch: 4, summary_loss: 0.28471, time: 93.37577


# [RESULT]: Train. Epoch: 0, summary_loss: 31.94577, time: 1479.94015
# [RESULT]: Val. Epoch: 0, summary_loss: 0.61383, time: 94.19924

####################################
# Fold: 1
# Fitter prepared. Device is cuda:1

# 2020-12-16T15:06:36.643936
# LR: 0.0002
# [RESULT]: Train. Epoch: 0, summary_loss: 31.92329, time: 4264.37266
# [RESULT]: Val. Epoch: 0, summary_loss: 0.45954, time: 169.30727

# 2020-12-16T16:20:31.988264
# LR: 0.0002
# [RESULT]: Train. Epoch: 1, summary_loss: 0.39175, time: 4285.17595
# [RESULT]: Val. Epoch: 1, summary_loss: 0.35737, time: 169.13260

# 2020-12-16T17:34:48.032052
# LR: 0.0002
# [RESULT]: Train. Epoch: 2, summary_loss: 0.32968, time: 4283.12818
# [RESULT]: Val. Epoch: 2, summary_loss: 0.31442, time: 169.27156

# 2020-12-16T18:49:02.099119
# LR: 0.0002
# [RESULT]: Train. Epoch: 3, summary_loss: 0.29665, time: 4286.38176
# [RESULT]: Val. Epoch: 3, summary_loss: 0.30096, time: 169.04031

# 2020-12-16T20:03:19.145026
# LR: 0.0002
# [RESULT]: Train. Epoch: 4, summary_loss: 0.27106, time: 4284.19108
# [RESULT]: Val. Epoch: 4, summary_loss: 0.30167, time: 169.36879

###################################################
# New version
# Fold: 1
# Fitter prepared. Device is cuda:1

# 2020-12-18T16:04:55.055592
# LR: 0.0002
# [RESULT]: Train. Epoch: 0, summary_loss: 32.99194, time: 1260.63828
# TP: 18, FP: 3105, FN: 3764, PRECISION: 0.0058, RECALL: 0.0048, F1 SCORE: 0.005213117923460117
# [RESULT]: Val. Epoch: 0, summary_loss: 0.71405, time: 86.69947

# 2020-12-18T16:27:24.152897
# LR: 0.0002
# [RESULT]: Train. Epoch: 1, summary_loss: 0.45980, time: 1279.22776
# TP: 85, FP: 7729, FN: 3697, PRECISION: 0.0109, RECALL: 0.0225, F1 SCORE: 0.014659788125129788
# [RESULT]: Val. Epoch: 1, summary_loss: 0.49883, time: 87.67601

# 2020-12-18T16:50:12.724554
# LR: 0.0002
# [RESULT]: Train. Epoch: 2, summary_loss: 0.34238, time: 1277.33376
# TP: 80, FP: 8393, FN: 3702, PRECISION: 0.0094, RECALL: 0.0212, F1 SCORE: 0.013055468825888171
# [RESULT]: Val. Epoch: 2, summary_loss: 0.47902, time: 87.64282

# 2020-12-18T17:12:59.367418
# LR: 0.0002
# [RESULT]: Train. Epoch: 3, summary_loss: 0.29125, time: 1278.55486
# TP: 69, FP: 6309, FN: 3713, PRECISION: 0.0108, RECALL: 0.0182, F1 SCORE: 0.01358220982190375
# [RESULT]: Val. Epoch: 3, summary_loss: 0.48011, time: 87.26458

# 2020-12-18T17:35:46.017510
# LR: 0.0002
# [RESULT]: Train. Epoch: 4, summary_loss: 0.25854, time: 1276.62195
# TP: 53, FP: 3015, FN: 3729, PRECISION: 0.0173, RECALL: 0.0140, F1 SCORE: 0.015473957998351695
# [RESULT]: Val. Epoch: 4, summary_loss: 0.49537, time: 86.90593

###################################################
Skfold
Fold: 1
Fitter prepared. Device is cuda:1

2020-12-19T03:23:13.688484
LR: 0.0002
[RESULT]: Train. Epoch: 0, summary_loss: 32.61389, time: 1247.27015
TP: 12, FP: 1818, FN: 3859, PRECISION: 0.0066, RECALL: 0.0031, F1 SCORE: 0.004209351884828015
[RESULT]: Val. Epoch: 0, summary_loss: 0.64355, time: 86.10779

2020-12-19T03:45:28.810166
LR: 0.0002
[RESULT]: Train. Epoch: 1, summary_loss: 0.47198, time: 1271.35743
TP: 64, FP: 7411, FN: 3807, PRECISION: 0.0086, RECALL: 0.0165, F1 SCORE: 0.011281059366908418
[RESULT]: Val. Epoch: 1, summary_loss: 0.37799, time: 86.76142

2020-12-19T04:08:08.665393
LR: 0.0002
[RESULT]: Train. Epoch: 2, summary_loss: 0.35497, time: 1273.21498
TP: 57, FP: 4161, FN: 3814, PRECISION: 0.0135, RECALL: 0.0147, F1 SCORE: 0.014092713939613513
[RESULT]: Val. Epoch: 2, summary_loss: 0.33371, time: 87.09410

2020-12-19T04:30:50.736873
LR: 0.0002
[RESULT]: Train. Epoch: 3, summary_loss: 0.30731, time: 1271.67354
TP: 63, FP: 5273, FN: 3808, PRECISION: 0.0118, RECALL: 0.0163, F1 SCORE: 0.013684752165351107
[RESULT]: Val. Epoch: 3, summary_loss: 0.30095, time: 87.13953

2020-12-19T04:53:31.246364
LR: 0.0002
[RESULT]: Train. Epoch: 4, summary_loss: 0.27455, time: 1272.93314
TP: 86, FP: 5669, FN: 3785, PRECISION: 0.0149, RECALL: 0.0222, F1 SCORE: 0.017867792588509148
[RESULT]: Val. Epoch: 4, summary_loss: 0.27812, time: 86.94070

SyntaxError: invalid syntax (<ipython-input-18-4683a633db8c>, line 100)